# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [148]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [149]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
499,YUM
500,ZBH
501,ZBRA
502,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [150]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url).json()
response

{'companyName': 'Apple Inc',
 'marketcap': 2425959559215,
 'week52high': 183.18,
 'week52low': 128.54,
 'week52highSplitAdjustOnly': 183.93,
 'week52lowSplitAdjustOnly': 124.87,
 'week52change': 0.164236774852804,
 'sharesOutstanding': 16562813464,
 'float': 0,
 'avg10Volume': 88610698,
 'avg30Volume': 112691768,
 'day200MovingAvg': 168.58,
 'day50MovingAvg': 157.12,
 'employees': 150459,
 'ttmEPS': 6.3,
 'ttmDividendRate': 0.9100342566988873,
 'dividendYield': 0.006338585142694167,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-25',
 'nextEarningsDate': '2022-07-26',
 'peRatio': 23.544516118171344,
 'beta': 1.3038695083024174,
 'maxChangePercent': 55.83276326634658,
 'year5ChangePercent': 3.12035351333173,
 'year2ChangePercent': 0.7922634746351406,
 'year1ChangePercent': 0.1628426324851295,
 'ytdChangePercent': -0.1807558160915169,
 'month6ChangePercent': -0.09969320598704169,
 'month3ChangePercent': -0.10799745077530437,
 'month1ChangePercent': -0.07244321026704773,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [151]:
response['year1ChangePercent']

0.1628426324851295

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [152]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [153]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        final_dataframe = pd.concat(
            [final_dataframe,
            pd.DataFrame([[
                symbol,
                response[symbol]['price'],
                response[symbol]['stats']['year1ChangePercent'],
                'N/A'
                ]], columns = my_columns
            )
            ],
            ignore_index=True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.97,-0.066284,N/A
1,AAL,16.6,-0.343828,N/A
2,AAP,200,0.010869,N/A
3,AAPL,151.17,0.16449,N/A
4,ABBV,146.63,0.370763,N/A
...,...,...,...,...
499,YUM,124.45,0.016235,N/A
500,ZBH,119.04,-0.244974,N/A
501,ZBRA,348.63,-0.363184,N/A
502,ZION,57.41,-0.027359,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [154]:
# inplace = True will modify the original dataframe
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,149,DVN,80.6,1.61991,N/A
1,354,OXY,72.63,1.485802,N/A
2,314,MRO,32.72,1.365383,N/A
3,121,CTRA,36.44,1.141909,N/A
4,40,APA,50.33,1.086429,N/A
5,110,COP,123.12,1.080131,N/A
6,178,FANG,158.96,0.847941,N/A
7,385,PXD,290.94,0.801984,N/A
8,91,CF,95.77,0.780541,N/A
9,468,VLO,143.84,0.736862,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [155]:
def input_portfolio_size():
    global portfolio_size 
    portfolio_size = input('Please enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print("Please enter a number!")
        portfolio_size = input('Please enter the size of your portfolio: ')
        float(portfolio_size)

input_portfolio_size()
print(portfolio_size)

1000000


In [156]:
# Calculate position size
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [198]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = pd.concat([
            hqm_dataframe, 
            pd.DataFrame(
                [[
                    symbol,
                    response[symbol]['price'],
                    'N/A',
                    response[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    response[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    response[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    response[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ]],
                columns=hqm_columns
            )
        ], ignore_index=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.75,N/A,-0.072614,N/A,-0.158943,N/A,-0.047104,N/A,0.056188,N/A,N/A
1,AAL,16.4,N/A,-0.342884,N/A,-0.092894,N/A,0.122655,N/A,-0.089861,N/A,N/A
2,AAP,195.66,N/A,0.011252,N/A,-0.16337,N/A,-0.072133,N/A,-0.059827,N/A,N/A
3,AAPL,146.32,N/A,0.171998,N/A,-0.115565,N/A,-0.105142,N/A,-0.072099,N/A,N/A
4,ABBV,149.9,N/A,0.370941,N/A,0.23553,N/A,-0.018757,N/A,-0.042474,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,121.66,N/A,0.02296,N/A,-0.067313,N/A,0.010596,N/A,0.040321,N/A,N/A
500,ZBH,118.76,N/A,-0.25368,N/A,-0.056036,N/A,-0.015138,N/A,-0.016638,N/A,N/A
501,ZBRA,344.01,N/A,-0.361441,N/A,-0.444953,N/A,-0.156066,N/A,-0.021241,N/A,N/A
502,ZION,57.53,N/A,-0.021436,N/A,-0.113638,N/A,-0.133377,N/A,0.018368,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [199]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percertile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percertile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.75,N/A,-0.072614,0.436508,-0.158943,0.327381,-0.047104,0.446429,0.056188,0.797619,N/A
1,AAL,16.4,N/A,-0.342884,0.071429,-0.092894,0.47619,0.122655,0.878968,-0.089861,0.047619,N/A
2,AAP,195.66,N/A,0.011252,0.575397,-0.16337,0.309524,-0.072133,0.373016,-0.059827,0.111111,N/A
3,AAPL,146.32,N/A,0.171998,0.837302,-0.115565,0.424603,-0.105142,0.28373,-0.072099,0.079365,N/A
4,ABBV,149.9,N/A,0.370941,0.938492,0.23553,0.892857,-0.018757,0.541667,-0.042474,0.174603,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,121.66,N/A,0.02296,0.593254,-0.067313,0.527778,0.010596,0.634921,0.040321,0.702381,N/A
500,ZBH,118.76,N/A,-0.25368,0.146825,-0.056036,0.549603,-0.015138,0.553571,-0.016638,0.30754,N/A
501,ZBRA,344.01,N/A,-0.361441,0.061508,-0.444953,0.021825,-0.156066,0.142857,-0.021241,0.279762,N/A
502,ZION,57.53,N/A,-0.021436,0.535714,-0.113638,0.430556,-0.133377,0.190476,0.018368,0.555556,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [200]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [201]:
hqm_dataframe.sort_values('HQM Score', ascending= False, inplace= True)
hqm_dataframe= hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [202]:
hqm_dataframe.rename(columns={"index": "Rank"})

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,32.64,N/A,1.359706,0.996032,1.026333,0.996032,0.336891,0.988095,0.122897,0.970238,0.987599
1,CTRA,36.37,N/A,1.152679,0.994048,0.893109,0.992063,0.411266,0.990079,0.12681,0.974206,0.987599
2,MPC,111.95,N/A,0.807271,0.986111,0.781376,0.986111,0.423904,0.992063,0.153414,0.982143,0.986607
3,DVN,77.8,N/A,1.609023,1.0,0.889808,0.990079,0.327395,0.986111,0.106697,0.948413,0.981151
4,APA,48.61,N/A,1.14725,0.992063,0.933874,0.994048,0.236922,0.972222,0.102947,0.938492,0.974206
5,VLO,140.1,N/A,0.772215,0.982143,1.043009,0.998016,0.64133,0.998016,0.086109,0.89881,0.969246
6,HAL,42.67,N/A,0.687132,0.972222,0.859615,0.988095,0.213744,0.962302,0.111501,0.952381,0.96875
7,OXY,70.97,N/A,1.405214,0.998016,1.394874,1.0,0.248648,0.974206,0.073189,0.857143,0.957341
8,COP,122.25,N/A,1.052245,0.990079,0.647381,0.978175,0.183353,0.940476,0.095534,0.920635,0.957341
9,BKR,39.39,N/A,0.500162,0.954365,0.578817,0.968254,0.139808,0.90873,0.183358,0.986111,0.954365


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [203]:
input_portfolio_size()

In [204]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,32.64,6127,1.359706,0.996032,1.026333,0.996032,0.336891,0.988095,0.122897,0.970238,0.987599
1,CTRA,36.37,5499,1.152679,0.994048,0.893109,0.992063,0.411266,0.990079,0.12681,0.974206,0.987599
2,MPC,111.95,1786,0.807271,0.986111,0.781376,0.986111,0.423904,0.992063,0.153414,0.982143,0.986607
3,DVN,77.8,2570,1.609023,1.0,0.889808,0.990079,0.327395,0.986111,0.106697,0.948413,0.981151
4,APA,48.61,4114,1.14725,0.992063,0.933874,0.994048,0.236922,0.972222,0.102947,0.938492,0.974206
5,VLO,140.1,1427,0.772215,0.982143,1.043009,0.998016,0.64133,0.998016,0.086109,0.89881,0.969246
6,HAL,42.67,4687,0.687132,0.972222,0.859615,0.988095,0.213744,0.962302,0.111501,0.952381,0.96875
7,OXY,70.97,2818,1.405214,0.998016,1.394874,1.0,0.248648,0.974206,0.073189,0.857143,0.957341
8,COP,122.25,1635,1.052245,0.990079,0.647381,0.978175,0.183353,0.940476,0.095534,0.920635,0.957341
9,BKR,39.39,5077,0.500162,0.954365,0.578817,0.968254,0.139808,0.90873,0.183358,0.986111,0.954365


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [217]:
writer = pd.ExcelWriter('Momentum_Strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name= 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [218]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1,
            'font_size': 14
        }
    )

In [219]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template ],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile',percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats:
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 30, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [220]:
writer.save()